# Assignment for Coursera_Capstone

### Import Necessary libraries

In [25]:
## import necessary libraries
import pandas as pd
import numpy as np
import sklearn 
import matplotlib as mpl
import requests
from bs4 import BeautifulSoup

### Scrape datasets from online
We need to present the datasets in the format of dataframe for further operations

In [26]:
## scrape datasets from website 
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content, 'lxml')
table = soup.find_all('table')[0]
Toronto_data_raw = pd.read_html(str(table))
Toronto_data1 = Toronto_data_raw[0]
Toronto_data1.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Set the column names

In [27]:
## clean the dataset and set it into a dataframe format
Toronto_data = Toronto_data1.drop([0],axis = 0)
Toronto_data = Toronto_data.reset_index()
Toronto_data.columns = ['index','Postcode', 'Borough','Neighborhood']
Toronto_data = Toronto_data.drop(['index'], axis = 1)
Toronto_data.head()

,Postcode,Borough,Neighborhood
0,M2A,Not assigned,Not assigned
1,M3A,North York,Parkwoods
2,M4A,North York,Victoria Village
3,M5A,Downtown Toronto,Harbourfront
4,M5A,Downtown Toronto,Regent Park


### Clean data with unassigned borough values

In [28]:
## Ignore rows with unassigned borough values 
Toronto_data = Toronto_data[Toronto_data['Borough'] != 'Not assigned']

Toronto_data.shape

(211, 3)

### Format data with same postcode but different neighborhoods

In [29]:
## use another dataframe for the operations
Toronto_try = Toronto_data

In [30]:
## Test if there are repeating rows in the dataframe
for i in range(211):
    for j in range(211):
        if i != j and Toronto_try.iloc[i]['Postcode'] ==  Toronto_try.iloc[j]['Postcode'] and Toronto_try.iloc[i]['Neighborhood'] == Toronto_try.iloc[j]['Neighborhood']:
            print (i,j)

In [31]:
## combine the neighborhoods for the same postcode value
for i in range(1, 211):
    if Toronto_try.iloc[i]['Postcode'] ==  Toronto_try.iloc[i-1]['Postcode'] and Toronto_try.iloc[i]['Neighborhood'] != Toronto_try.iloc[i-1]['Neighborhood']:
        Toronto_try.iloc[i]['Neighborhood'] = str(Toronto_try.iloc[i-1]['Neighborhood']) + ', ' + Toronto_try.iloc[i]['Neighborhood'] 

## reset index 
Toronto_try1 = Toronto_try.reset_index()
Toronto_try1 = Toronto_try1.drop(['index'], axis = 1)

## drop redundant rows
redundant_rows = []
for i in range(1, 211):
    if Toronto_try1.iloc[i]['Postcode'] == Toronto_try1.iloc[i-1]['Postcode']:
        redundant_rows.append(i-1) 

## reset index for the new dataframe again
Toronto_try2 = Toronto_try1.drop(redundant_rows)
Toronto_try2 = Toronto_try2.reset_index()  
Toronto_try2 = Toronto_try2.drop(['index'], axis = 1)
Toronto_try2.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Not assigned


### Format missing data of neighborhood with borough value

In [32]:
## use row iterations to fill in the missing data of neighborhood
for index, row in Toronto_try2.iterrows():
    if row['Neighborhood'] == 'Not assigned':
        row['Neighborhood'] = row['Borough']
## use another explainative dataframe name to represent the final data 
Toronto_final = Toronto_try2
Toronto_final.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


### Shape of the dataframe 

In [33]:
Toronto_final.shape

(103, 3)

### Obtain Postcode Data (Second Qn Week 3)
Since the code involving geocoder takes too long to run and frequently encounters errors, we are extracting postcode data from the csv file provided

In [34]:
## Extract postcode data from csv files
postcode = pd.read_csv("http://cocl.us/Geospatial_data")
postcode.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Merge Toronto_final data and postcode data into one dataframe

In [35]:
## Test on the shape of the postal code data
## make sure it fits the original Toronto_final dataframe
postcode.shape

(103, 3)

In [36]:
## Add two columns ('Latitide', 'Longitude') to the dataframe 
Toronto_final.insert(3, "Latitude", np.zeros(103), True)
Toronto_final.insert(4, 'Longitude', np.zeros(103), True)
Toronto_final.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,0.0,0.0
1,M4A,North York,Victoria Village,0.0,0.0
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",0.0,0.0
3,M6A,North York,"Lawrence Heights, Lawrence Manor",0.0,0.0
4,M7A,Queen's Park,Queen's Park,0.0,0.0


In [52]:
## Use iterations to combine data from two dataframes
for index, row in Toronto_final.iterrows():
    for j in range(103):
        if row['Postcode'] == postcode.iloc[j]['Postal Code']:
            Toronto_final.at[index, 'Latitude'] = postcode.iloc[j]['Latitude']
            Toronto_final.at[index, 'Longitude'] = postcode.iloc[j]['Longitude']
Toronto_final.head(10)          

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
